In [28]:
from imutils import paths
import numpy as np
import argparse
import imutils
import cv2
import matplotlib.pyplot as mp

# Reading the Images


In [29]:
image = cv2.imread("0.jpg")

cv2.imshow("",image)
cv2.waitKey()
cv2.destroyAllWindows()

# Extracting the Passport from the Background


In [30]:
# loop over the contours
b=0 # i am using this variable as a boolean, if the right contour is found then it will equal to 1 if not then it stays as Zero
rec=250
for i in range(0,4):  # this loop is to checks the image from 4 different perspective by flipping it 4 time as the passport in the image my be flipped, so we need to correct that

    rectKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (rec, rec))  
    # here we treated the rect as a square as the target outout's shape (Passport) will most likely be colse to a square
    sqKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (200, 200))  # sqKernel is used to fill the gap between the 2 sentences we want to extract(MRZ)
    image = imutils.resize(image, height=600)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (3, 3), 0)
    blackhat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, rectKernel)
    gradX = cv2.Sobel(blackhat, ddepth=cv2.CV_32F, dx=1, dy=0, ksize=-1)
    gradX = np.absolute(gradX)
    (minVal, maxVal) = (np.min(gradX), np.max(gradX))
    gradX = (255 * ((gradX - minVal) / (maxVal - minVal))).astype("uint8")
    gradX = cv2.morphologyEx(gradX, cv2.MORPH_CLOSE, rectKernel)
    thresh = cv2.threshold(gradX, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, sqKernel)
    thresh = cv2.erode(thresh, None, iterations=4)
    p = int(image.shape[1] * 0.05)
    thresh[:, 0:p] = 0
    thresh[:, image.shape[1] - p:] = 0
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
              cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
    for c in cnts:
            # compute the bounding box of the contour and use the contour to
            # compute the aspect ratio and coverage ratio of the bounding box
            # width to the width of the image
            (x, y, w, h) = cv2.boundingRect(c)
            ar = w / float(h)
            crWidth = w / float(gray.shape[1])
            # check to see if the aspect ratio and coverage width are within
            # acceptable criteria
            if ar > 0 and crWidth > 0.6 : 

              # extract the ROI from the image and draw a bounding box
              # surrounding the MRZ
                passport = image[y:y + h, x:x + w].copy()
                #cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                b=1
                break
    if b==1: break
    if b == 0: # that means we didn't find the right contour, and we might need to flipp the image and start all over again
              image = cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)
   
# show  images
cv2.imshow("The Original Image",image)
cv2.imshow("Passport",passport)
cv2.waitKey()
cv2.destroyAllWindows()

# Expanding the ROI
There is a chance that the algorithm may cut a small wanted details from the passport when extracting it. So, "expandingROI" function here tries to decrease this chance

In [31]:
   # I wanted to increase the width by 32 degree from both sides, and the height by just 8 degrees
  # This function here makes sure that I am not going out of the boundries of the image
def expandingROI(image,x,y,w,h,increasingWidhParameter,increasingHeightParameter):
    x1=x
    x2=x+w
    y1=y
    y2=y+h
    for i in range (0,4):  
        if x1>=increasingWidhParameter:        
              x1=x1-increasingWidhParameter
        if x2+increasingWidhParameter<image.shape[1]:
              x2=x2+increasingWidhParameter
        if y1>=increasingHeightParameter:
              y1=y1-increasingHeightParameter
        if y2+increasingHeightParameter< image.shape[0]:
              y2=y2+increasingHeightParameter
    ROI = image[y1:y2, x1:x2].copy()
    return ROI

passport= expandingROI(image,x,y,w,h,6,2)

cv2.imshow("ROI",passport)
cv2.waitKey()
cv2.destroyAllWindows()



# Visualizing the Result with Clustering Segmentation with Kmeans 
 Here I wanted to Visualize how well the passport is separated from the background and also do some sort of threshold to help in the next phase when extractin the MRZ

In [32]:
segmentedImage= passport.copy()
temp = cv2.cvtColor(segmentedImage, cv2.COLOR_BGR2RGB)
pixel_values = temp.reshape((-1, 3))
# convert to float
pixel_values = np.float32(pixel_values)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 500, 0.2)
k = 2
_, labels, (centers) = cv2.kmeans(pixel_values, k, None, criteria, 20, cv2.KMEANS_RANDOM_CENTERS)
centers = np.uint8(centers)

# flatten the labels array
labels = labels.flatten()
cntr0=0
cntr1=0
for k in range(len(labels)):   # here i am trying to figure out wich label is the dominant assuming that it has to be the passport to then remove the other label from the image
    if labels[k]==0:
        cntr0+=1
    else:
        cntr1+=1
segmented_image = centers[labels.flatten()]
segmented_image = segmented_image.reshape(segmentedImage.shape)
masked_image = np.copy(segmentedImage)
# convert to the shape of a vector of pixel values
masked_image = masked_image.reshape((-1, 3))
# color (i.e cluster) to disable
cluster = 0
if cntr0> cntr1: cluster= 1
else: cluster = 0
masked_image[labels == cluster] = [0, 0, 0]
masked_image = masked_image.reshape(segmentedImage.shape)


# show the image
cv2.imshow('Segmanted Image',masked_image)
cv2.waitKey()
cv2.destroyAllWindows()


# Extracting the MRZ

In [33]:
# loop over the contours
b=0 # i am using this variable as a boolean, if the right contour is found then it will equal to 1 if not then it stays as Zero
recWidth=32
recHeight=15
sq=25
image=masked_image
for j in range(0,7):  #this loop is basically for trying different sizes for the kernels to tune it so we be able to extract the right Contour
  for i in range(0,4):  # this loop is to checks the image from 4 different perspective by flipping it 4 time as the passport in the image my be flipped, so we need to correct that
      rectKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (recWidth, recHeight))
      sqKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (sq, sq))
      image = imutils.resize(image, height=600)
      passport = imutils.resize(passport, height=600)
      gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
      gray = cv2.GaussianBlur(gray, (3, 3), 0)
      blackhat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, rectKernel)
      gradX = cv2.Sobel(blackhat, ddepth=cv2.CV_32F, dx=1, dy=0, ksize=-1)
      gradX = np.absolute(gradX)
      (minVal, maxVal) = (np.min(gradX), np.max(gradX))
      gradX = (255 * ((gradX - minVal) / (maxVal - minVal))).astype("uint8")
      gradX = cv2.morphologyEx(gradX, cv2.MORPH_CLOSE, rectKernel)
      thresh = cv2.threshold(gradX, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
      thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, sqKernel)
      thresh = cv2.erode(thresh, None, iterations=4)
      p = int(image.shape[1] * 0.05)
      thresh[:, 0:p] = 0
      thresh[:, image.shape[1] - p:] = 0
      cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
                cv2.CHAIN_APPROX_SIMPLE)
      cnts = imutils.grab_contours(cnts)
      cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
      for c in cnts:
              # compute the bounding box of the contour and use the contour to
              # compute the aspect ratio and coverage ratio of the bounding box
              # width to the width of the image
              (x, y, w, h) = cv2.boundingRect(c)
              ar = w / float(h)
              crWidth = w / float(gray.shape[1])
              width=w
              height=h
              # check to see if the aspect ratio and coverage width are within
              # acceptable criteria
            # if the height is too big we need to decrease the width of the rectangle kernel to extract smaller contours
              if h>=150 and recWidth >5:
                recWidth= recWidth-5
                continue
              if ar > 2 and crWidth > 0.5 :
                # pad the bounding box since we applied erosions and now need
                # to re-grow it
                  pX = int((x + w) * 0.03)
                  pY = int((y + h) * 0.03)
                  (x, y) = (x - pX, y - pY)
                  (w, h) = (w + (pX * 2), h + (pY * 2))

                # extract the ROI from the image and draw a bounding box
                  roi= expandingROI(passport,x,y,w,h,3,1)

                  #cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

                  b=1
                  break
      if b==1: break

      if b == 0:  # that means we didn't find the right contour, and we might need to flipp the image
                image = cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)
                passport = cv2.rotate(passport, cv2.ROTATE_90_COUNTERCLOCKWISE)

  if width>= 12*height:    # if the width is so much bigger than the height that means we need to start all over again
                                         # as our square kernel needs to be bigger to fill the gaps between the 2 sentences
                  sq=sq+5
  else: break


   
cv2.imshow("",image)
cv2.imshow("MRZ",roi)
cv2.waitKey()
cv2.destroyAllWindows()

# Reading the MRZ
for this phase, I have used OCR (pytesseract) to read the MRZ and extract it as a text file


In [34]:
import pytesseract

In [35]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'


In [42]:
kernel = np.ones((5, 5), np.uint8)
dilated =cv2.dilate(roi, kernel, 5)


In [46]:
cont=cv2.convertScaleAbs(roi, 12, 1)
gray=cv2.cvtColor(cont,cv2.COLOR_BGR2GRAY)
thresh = cv2.threshold(gray, 0, 255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]




In [47]:
text=pytesseract.image_to_string(thresh,config='--psm 6')
text =text.split('\n')
print(text[0][0:25],'\n',text[1])

j PEAUSC ETI ZEWK< SANE C 
 PE09 15986 14584060771 1903278< 406009 96K< <<< 04


In [43]:
cv2.imshow("MRZ",roi)
cv2.waitKey()
cv2.destroyAllWindows()

In [45]:
text=pytesseract.image_to_string(roi
                                 ,config='--psm 6')
text =text.split('\n')
print(text[0][0:25],'\n',text[1])

E PSAUSCITIZENK< JANE <<  
 PE09159861AUS8406077F 1903278<40600996K<<<<0%


In [326]:
textFile = open('sample2.txt', 'a')
textFile.write(text[0][0:25]+'\n'+text[1])
textFile.close()